# --- Day 21: RPG Simulator 20XX ---

Little Henry Case got a new video game for Christmas. It's an **RPG**, and he's stuck on a boss. He needs to know what equipment to buy at the shop. He hands you the **controller**.

In this game, the player (you) and the enemy (the boss) take turns attacking. The player always goes first. Each attack reduces the opponent's hit points by at least `1`. The first character at or below `0` hit points loses.

Damage dealt by an attacker each turn is equal to the attacker's damage score minus the defender's armor score. An attacker always does at least `1` damage. So, if the attacker has a damage score of `8`, and the defender has an armor score of `3`, the defender loses `5` hit points. If the defender had an armor score of `300`, the defender would still lose `1` hit point.

Your damage score and armor score both start at zero. They can be increased by buying items in exchange for gold. You start with no items and have as much gold as you need. Your total damage or armor is equal to the sum of those stats from all of your items. You have **100 hit points**.

Here is what the item shop is selling:

```
Weapons:    Cost  Damage  Armor
Dagger        8     4       0
Shortsword   10     5       0
Warhammer    25     6       0
Longsword    40     7       0
Greataxe     74     8       0

Armor:      Cost  Damage  Armor
Leather      13     0       1
Chainmail    31     0       2
Splintmail   53     0       3
Bandedmail   75     0       4
Platemail   102     0       5

Rings:      Cost  Damage  Armor
Damage +1    25     1       0
Damage +2    50     2       0
Damage +3   100     3       0
Defense +1   20     0       1
Defense +2   40     0       2
Defense +3   80     0       3
```

You must buy exactly one weapon; no dual-wielding. Armor is optional, but you can't use more than one. You can buy 0-2 rings (at most one for each hand). You must use any items you buy. The shop only has one of each item, so you can't buy, for example, two rings of Damage +3.

For example, suppose you have `8` hit points, `5` damage, and `5` armor, and that the boss has `12` hit points, `7` damage, and `2` armor:

+ The player deals `5-2 = 3` damage; the boss goes down to 9 hit points.
+ The boss deals `7-5 = 2` damage; the player goes down to 6 hit points.
+ The player deals `5-2 = 3` damage; the boss goes down to 6 hit points.
+ The boss deals `7-5 = 2` damage; the player goes down to 4 hit points.
+ The player deals `5-2 = 3` damage; the boss goes down to 3 hit points.
+ The boss deals `7-5 = 2` damage; the player goes down to 2 hit points.
+ The player deals `5-2 = 3` damage; the boss goes down to 0 hit points.

In this scenario, the player wins! (Barely.)

You have **100 hit points**. The boss's actual stats are in your puzzle input. What is the **least amount of gold you can spend** and still **win** the fight?

In [4]:
from collections import namedtuple
from copy import deepcopy
from itertools import combinations

boss = {
    'hp' : 100,
    'damage' : 8,
    'armor' : 2
}

Item = namedtuple('Item', ['name', 'cost', 'damage', 'armor'])

WEAPONS = [
    Item('Dagger',        8,     4,       0),
    Item('Shortsword',   10,     5,       0),
    Item('Warhammer',    25,     6,       0),
    Item('Longsword',    40,     7,       0),
    Item('Greataxe',     74,     8,       0),
]
ARMOR = [
    Item('Leather',      13,     0,       1),
    Item('Chainmail',    31,     0,       2),
    Item('Splintmail',   53,     0,       3),
    Item('Bandedmail',   75,     0,       4),
    Item('Platemail',   102,     0,       5),
]

RINGS = [
    Item('Damage +1',    25,     1,       0),
    Item('Damage +2',    50,     2,       0),
    Item('Damage +3',   100,     3,       0),
    Item('Defense +1',   20,     0,       1),
    Item('Defense +2',   40,     0,       2),
    Item('Defense +3',   80,     0,       3),
]
setups = []

# weapons only                = 5
# weapons + 1 armour          = 25
# weapons + 1 armour + 1 ring = 150
# weapons + 1 ring	          = 30
# weapons + 2 rings	          = 75
# weapons + 1 armour + 2rings = 375
# total                       = 660

for weapon in WEAPONS:
    # add weapon only (5)
    setups.append({
        'weapon' : weapon
    })

    # add weapon + armour (25)
    weaponsArmour = []
    for a in ARMOR:
        weaponsArmour.append({
            'weapon' : weapon,
            'armor'  : a
        })
    
    for wa in weaponsArmour:
        setups.append(wa)
    
    # add weapon + 1 ring (30)
    for r in RINGS:
        setups.append({
            'weapon' : weapon,
            'rings' : [r]
        })
    
    # weapons + 1 armour + 1 ring	(150)
    weaponsArmour1Ring = []
    for wa in weaponsArmour:
        for r in RINGS:
            weaponsArmour1Ring.append({
                'weapon' : wa['weapon'],
                'armor'  : wa['armor'],
                'rings'   : [r]
            })

    for wa1r in weaponsArmour1Ring:
        setups.append(wa1r)

    for r2 in list(combinations(RINGS, 2)):
        setups.append({
            'weapon' : weapon,
            'rings' : list(r2)
        })

        for a in ARMOR:
            setups.append({
                'weapon' : weapon,
                'armor' : a,
                'rings' : list(r2)
            })

for s in setups:   
    totals = {}

    # every setup has a weapon therefore set as default
    totals['cost'] = s['weapon'].cost
    totals['damage'] = s['weapon'].damage
    totals['armor'] = s['weapon'].armor

    if 'armor' in s:
        totals['cost'] += s['armor'].cost
        totals['damage'] += s['armor'].damage
        totals['armor'] += s['armor'].armor
    
    if 'rings' in s:
        for r in s['rings']:
            totals['cost'] += r.cost
            totals['damage'] += r.damage
            totals['armor'] += r.armor

    s['totals'] = totals

lowest_cost_winner = {}
highest_cost_loser = {}

for s in setups:
    current_boss = deepcopy(boss)
    user_health = 100

    while current_boss['hp'] > 0 and user_health > 0 :
        attack = s['totals']['damage'] - current_boss['armor']
        
        if attack > 0:
            current_boss['hp'] -= attack
        else:
            attack = 1
            current_boss['hp'] -= attack

        if current_boss['hp'] <= 0:
            break
        
        attack = current_boss['damage'] - s['totals']['armor']
        
        if attack > 0:
            user_health -= attack
        else:
            attack = 1
            user_health -= 1
        
    if user_health > current_boss['hp']:  
        if len(lowest_cost_winner.keys()) == 0:
            lowest_cost_winner = s
        elif s['totals']['cost'] < lowest_cost_winner['totals']['cost']:
            lowest_cost_winner = s
    else:
        if len(highest_cost_loser.keys()) == 0:
            highest_cost_loser = s
        elif s['totals']['cost'] > highest_cost_loser['totals']['cost']:
            highest_cost_loser = s
        

print('Part 1 : {}'.format(lowest_cost_winner['totals']['cost']))

Part 1 : 91


## --- Part Two ---
Turns out the shopkeeper is working with the boss, and can persuade you to buy whatever items he wants. The other rules still apply, and he still only has one of each item.

What is the **most** amount of gold you can spend and still **lose** the fight?

In [5]:
print('Part 2 : {}'.format(highest_cost_loser['totals']['cost']))

Part 2 : 158
